In [ ]:
#from google.colab import files
#from google.colab import drive

#drive.mount("/content/gdrive", force_remount=True)
#location = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


# FiltrarDatos:

In [2]:
from datetime import date
import pandas as pd


In [3]:
location = "/home/brian/Documents/udesa/pgvd/TP/recommendantion_system/Data/"

In [20]:
location = "/home/brian/Documents/udesa/pgvd/TP/recommendantion_system/Data/"
df_ads_views = pd.read_csv(location+'Raw/ads_views.csv')
df_advertiser_ids = pd.read_csv(location+'Raw/advertiser_ids.csv')
df_product_views = pd.read_csv(location+'Raw/product_views.csv')

In [21]:
df_ads_views.head()

,advertiser_id,product_id,type,date
0,6X20RDH567MX2X3TXYJ7,lmjpey,impression,2023-04-01
1,OAGTYWN8WFC997VLDJH7,3mx27r,impression,2023-04-01
2,P41C5HK4P2G5GFRMT6ZA,fprozu,impression,2023-04-01
3,62FIK8F2YT8JSFDBLEC9,s3am3r,impression,2023-04-01
4,1OW4LWJSBJK6E1D9FDH7,s8i4mp,impression,2023-04-01


In [22]:
df_product_views.head()

,advertiser_id,product_id,date
0,2WPF1NXECF3G6NUMWDO7,ua4slr,2023-04-01
1,03KNVBO915KY2ZPGA57J,gf3odv,2023-04-01
2,IDOFCO721HTJGDH7332G,88i2x0,2023-04-01
3,EN1SA43DTN2LIR8DEW5S,wgm1iv,2023-04-01
4,M0LU6DCI1WILGQBZ6808,arbc4z,2023-04-01


In [23]:
df_advertiser_ids.head()

,advertiser_id
0,IDOFCO721HTJGDH7332G
1,LW045DVYSGRD75TK6U54
2,SOVPFK3BBWKTQM9HOHWJ
3,M0LU6DCI1WILGQBZ6808
4,AK81O7W3KGPEN8LABG2N


In [24]:
def FiltrarDatos(df_advertiser_ids, df_product_views, df_ads_views, ds, **kwargs):
  '''
  funcion que agarra los logs de vistas de cada advertiser, de cada producto,
   y los filtra según la fecha de corrida del script y los advertisers activos
  '''


  #establecemos la fecha de corrida
  fecha_hoy = ds
  
  #convertimos los campos date en datetime
  df_product_views['date'] = pd.to_datetime(df_product_views['date'])
  df_ads_views['date'] = pd.to_datetime(df_ads_views['date'])
  
  #filtramos los dataframes para quedarnos con los datos antiguos a la fecha de hoy
  df_product_views = df_product_views[df_product_views['date']<=fecha_hoy]
  df_ads_views = df_ads_views[df_ads_views['date']<=fecha_hoy]
  
  #filtramos los datasets para quedarnos con los advertisers activos
  df_product_views = df_product_views[df_product_views['advertiser_id'].isin(df_advertiser_ids['advertiser_id'])]
  df_ads_views = df_ads_views[df_ads_views['advertiser_id'].isin(df_advertiser_ids['advertiser_id'])] 

  #Guardamos los DF filtrados
  df_product_views.to_csv(location+"/Processed/product_views_filt", index=False)
  df_ads_views.to_csv(location+"/Processed/ads_views_filt", index=False)


  return df_product_views, df_ads_views

In [25]:
df_product_views, df_ads_views = FiltrarDatos(df_advertiser_ids, df_product_views, df_ads_views)


# TopProduct

Por cada advertirser activo debe
devolver los 20 (o menos) productos más vistos en la web del cliente.

In [27]:
df_product_views_filt = pd.read_csv(location+'Processed/product_views_filt')
df_product_views_filt.head(5)

,advertiser_id,product_id,date
0,2WPF1NXECF3G6NUMWDO7,ua4slr,2023-04-01
1,IDOFCO721HTJGDH7332G,88i2x0,2023-04-01
2,EN1SA43DTN2LIR8DEW5S,wgm1iv,2023-04-01
3,M0LU6DCI1WILGQBZ6808,arbc4z,2023-04-01
4,LW045DVYSGRD75TK6U54,qvwvkh,2023-04-01


In [28]:
def TopProduct (df_product_views_filt), ds, **kwargs):
    '''
    Esta función toma las vistas de productos ya filtradas y por cada advertiser
    se queda con el top 20 de productos vistos
    '''
    #Agrupamos por advertiser y producto y contamos la cantidad de registros por cada combinación guardando el dato en la columna 'cantidad'
    df_count = df_product_views_filt.groupby(['advertiser_id', 'product_id']).size().reset_index(name='cantidad')

    #Ordenamos el DF anterior en order ascendente de vendedores y y de mayor a menor en cuanto a la cantidad.
    df_count_sorted = df_count.sort_values(['advertiser_id', 'cantidad'], ascending=[True, False])

    #Con el DF ordenado agrupamos por advertiser y tomamos el top 20.
    df_top20 = df_count_sorted.groupby('advertiser_id').head(20)
    
    #Creamos una columna con la fecha de recomendacion
    fecha_hoy = ds

    df_top20['fecha_recom'] = fecha_hoy

    df_top20.to_csv(location+"/Processed/TopProduct", index=False)

    return df_top20

In [29]:
TopProduct(df_product_views_filt)

,advertiser_id,product_id,cantidad
30,2WPF1NXECF3G6NUMWDO7,boj69r,22
10,2WPF1NXECF3G6NUMWDO7,45fvr1,21
50,2WPF1NXECF3G6NUMWDO7,igiyhv,21
51,2WPF1NXECF3G6NUMWDO7,j368cq,20
81,2WPF1NXECF3G6NUMWDO7,ua4slr,20
...,...,...,...
1930,Y0W3K7OV6ZLILW96OO3K,dinjn2,19
1936,Y0W3K7OV6ZLILW96OO3K,f7cuiv,19
1940,Y0W3K7OV6ZLILW96OO3K,gnxntg,19
1959,Y0W3K7OV6ZLILW96OO3K,oc20y8,19


In [ ]:
#df_top20.loc[df_top20['advertiser_id']=='5E325T5HYL61QSABVR5V']

# TopCTR:

Por cada advertirser activo debe
devolver los 20 (o menos) productos con mejor click-through-rate.

In [30]:
df_ads_views_filt = pd.read_csv(location+'Processed/ads_views_filt')
df_ads_views_filt.head(5)

,advertiser_id,product_id,type,date
0,6X20RDH567MX2X3TXYJ7,lmjpey,impression,2023-04-01
1,OAGTYWN8WFC997VLDJH7,3mx27r,impression,2023-04-01
2,P41C5HK4P2G5GFRMT6ZA,fprozu,impression,2023-04-01
3,62FIK8F2YT8JSFDBLEC9,s3am3r,impression,2023-04-01
4,OAGTYWN8WFC997VLDJH7,osyamx,impression,2023-04-01


In [44]:
def TopCTR (df_ads_views_filt, ds, **kwargs)):
    # Agrupamos el DF por advertiser, producto y tipo para luego contar la cantidad de veces que aparece cada combinación
    df_count = df_ads_views_filt.groupby(['advertiser_id', 'product_id', 'type']).size().reset_index(name='cantidad')

    # Pivoteamos el DF para tener una fila por cada combinación de adviser-producto y columnas para la cantidad de vistas y clicks
    df_pivoted = df_count.pivot_table(index=['advertiser_id', 'product_id'], columns='type', values='cantidad', fill_value=0).reset_index()

    # Filtramos el DF para eliminar combinaciones donde no hay clicks
    df_filtered = df_pivoted[df_pivoted['click'] > 0]
    
    # Calculamos la tasa de click para cada combinación de advertiser-product
    df_filtered['click-through-rate'] = df_filtered['click'] / df_pivoted['impression']

    # Ordenamos el DF por advertiser en forma ascendente y rate descendente.
    df_sorted = df_filtered.sort_values(['advertiser_id', 'click-through-rate'], ascending=[True, False])

    # Agrupamos el DF por advertiser y tomamos los top 20 productos con mejor tasa para cada advertiser.
    df_top20_CTR = df_sorted.groupby('advertiser_id').head(20)
    
    #Creamos una columna con la fecha de recomendacion
    fecha_hoy = ds
    df_top20_CTR['fecha_recom'] = ds #pd.to_datetime(pd.Timestamp.today().date()).strftime('%Y-%m-%d')

    df_top20_CTR.to_csv(location+"/Processed/TopCTR", index=False)

    return df_top20_CTR

In [61]:
from datetime import timedelta
pd.to_datetime(pd.Timestamp.today().date()) - timedelta(days=1)

Timestamp('2023-04-21 00:00:00')

In [64]:
from datetime import datetime

ds_str = '2023-04-22'
ds_datetime = datetime.strptime(ds_str, '%Y-%m-%d') - timedelta(days=1)

print(ds_datetime)

2023-04-21 00:00:00


In [45]:
df_top20_CTR = TopCTR (df_ads_views_filt)

/tmp/ipykernel_50023/2027873570.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['click-through-rate'] = df_filtered['click'] / df_pivoted['impression']
/tmp/ipykernel_50023/2027873570.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top20_CTR['fecha_recom'] = pd.to_datetime(pd.Timestamp.today().date()).strftime('%Y-%m-%d')


In [46]:
df_top20_CTR.head()

type,advertiser_id,product_id,click,impression,click-through-rate,fecha_recom
52,2WPF1NXECF3G6NUMWDO7,jarowj,1,9,0.111111,2023-04-22
4,2WPF1NXECF3G6NUMWDO7,10kluo,1,10,0.100000,2023-04-22
22,2WPF1NXECF3G6NUMWDO7,93h00e,1,10,0.100000,2023-04-22
67,2WPF1NXECF3G6NUMWDO7,o80z1x,1,10,0.100000,2023-04-22
33,2WPF1NXECF3G6NUMWDO7,dwknv4,1,13,0.076923,2023-04-22


## Probando la función

In [ ]:
df_count = df_ads_views_filt.groupby(['advertiser_id', 'product_id', 'type']).size().reset_index(name='cantidad')
df_count

,advertiser_id,product_id,type,cantidad
0,2WPF1NXECF3G6NUMWDO7,01mgr3,click,1
1,2WPF1NXECF3G6NUMWDO7,01mgr3,impression,12
2,2WPF1NXECF3G6NUMWDO7,0299ko,impression,11
3,2WPF1NXECF3G6NUMWDO7,18v6fo,impression,21
4,2WPF1NXECF3G6NUMWDO7,1ch7g8,impression,16
...,...,...,...,...
2249,Y0W3K7OV6ZLILW96OO3K,xg3tns,impression,18
2250,Y0W3K7OV6ZLILW96OO3K,y2y3fc,impression,13
2251,Y0W3K7OV6ZLILW96OO3K,yti917,impression,14
2252,Y0W3K7OV6ZLILW96OO3K,z8otxw,impression,18


In [ ]:
# Pivoteamos el DF para tener una fila por cada combinación de adviser-producto y columnas para la cantidad de vistas y clicks
df_pivoted = df_count.pivot_table(index=['advertiser_id', 'product_id'], columns='type', values='cantidad', fill_value=0).reset_index()
df_pivoted

type,advertiser_id,product_id,click,impression
0,2WPF1NXECF3G6NUMWDO7,01mgr3,1,12
1,2WPF1NXECF3G6NUMWDO7,0299ko,0,11
2,2WPF1NXECF3G6NUMWDO7,18v6fo,0,21
3,2WPF1NXECF3G6NUMWDO7,1ch7g8,0,16
4,2WPF1NXECF3G6NUMWDO7,1gegxk,0,12
...,...,...,...,...
1995,Y0W3K7OV6ZLILW96OO3K,xg3tns,0,18
1996,Y0W3K7OV6ZLILW96OO3K,y2y3fc,0,13
1997,Y0W3K7OV6ZLILW96OO3K,yti917,0,14
1998,Y0W3K7OV6ZLILW96OO3K,z8otxw,0,18


In [ ]:
# Filtramos el DF para eliminar combinaciones donde no hay clicks
df_filtered = df_pivoted[df_pivoted['click'] > 0]

In [ ]:
# Calculamos la tasa de click para cada combinación de 'vendedor' y 'producto'
df_filtered['click-through-rate'] = df_filtered['click'] / df_filtered['impression']
df_filtered

<ipython-input-58-1cd20f8acec2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['click-through-rate'] = df_filtered['click'] / df_filtered['impression']


type,advertiser_id,product_id,click,impression,click-through-rate
0,2WPF1NXECF3G6NUMWDO7,01mgr3,1,12,0.083333
9,2WPF1NXECF3G6NUMWDO7,2dklx7,1,21,0.047619
14,2WPF1NXECF3G6NUMWDO7,4miw6l,1,9,0.111111
22,2WPF1NXECF3G6NUMWDO7,6m92v0,2,7,0.285714
26,2WPF1NXECF3G6NUMWDO7,8ldyn6,1,11,0.090909
...,...,...,...,...,...
1953,Y0W3K7OV6ZLILW96OO3K,ipebka,1,15,0.066667
1980,Y0W3K7OV6ZLILW96OO3K,scgza4,1,13,0.076923
1981,Y0W3K7OV6ZLILW96OO3K,szo5zy,1,19,0.052632
1984,Y0W3K7OV6ZLILW96OO3K,tuvn4k,1,18,0.055556


In [ ]:
# Ordenamos el DataFrame por 'vendedor' y 'tasa' descendente
df_sorted = df_filtered.sort_values(['advertiser_id', 'click-through-rate'], ascending=[True, False])
df_sorted


type,advertiser_id,product_id,click,impression,click-through-rate
22,2WPF1NXECF3G6NUMWDO7,6m92v0,2,7,0.285714
14,2WPF1NXECF3G6NUMWDO7,4miw6l,1,9,0.111111
38,2WPF1NXECF3G6NUMWDO7,csdo4h,1,9,0.111111
26,2WPF1NXECF3G6NUMWDO7,8ldyn6,1,11,0.090909
71,2WPF1NXECF3G6NUMWDO7,nljn8x,1,11,0.090909
...,...,...,...,...,...
1953,Y0W3K7OV6ZLILW96OO3K,ipebka,1,15,0.066667
1918,Y0W3K7OV6ZLILW96OO3K,6b2ov5,1,17,0.058824
1984,Y0W3K7OV6ZLILW96OO3K,tuvn4k,1,18,0.055556
1981,Y0W3K7OV6ZLILW96OO3K,szo5zy,1,19,0.052632


In [ ]:
# Agrupamos el DataFrame por 'vendedor' y tomamos los top 20 productos con mejor tasa para cada vendedor
df_top20 = df_sorted.groupby('advertiser_id').head(20)
df_top20

type,advertiser_id,product_id,click,impression,click-through-rate
22,2WPF1NXECF3G6NUMWDO7,6m92v0,2,7,0.285714
14,2WPF1NXECF3G6NUMWDO7,4miw6l,1,9,0.111111
38,2WPF1NXECF3G6NUMWDO7,csdo4h,1,9,0.111111
26,2WPF1NXECF3G6NUMWDO7,8ldyn6,1,11,0.090909
71,2WPF1NXECF3G6NUMWDO7,nljn8x,1,11,0.090909
...,...,...,...,...,...
1953,Y0W3K7OV6ZLILW96OO3K,ipebka,1,15,0.066667
1918,Y0W3K7OV6ZLILW96OO3K,6b2ov5,1,17,0.058824
1984,Y0W3K7OV6ZLILW96OO3K,tuvn4k,1,18,0.055556
1981,Y0W3K7OV6ZLILW96OO3K,szo5zy,1,19,0.052632


# Funcion DBWriting

In [34]:
df_topCTR = pd.read_csv(location+"/Processed/TopCTR")
df_topProduct = pd.read_csv(location+"/Processed/TopProduct")

In [47]:
def DBWriting(df_topCTR, df_topProduct):
    
    df_topCTR.to_csv(location + 'RDS/df_topCTR.csv', index=False)
    df_topProduct.to_csv(location + 'RDS/df_topProduct.csv', index=False)

    return df_topCTR, df_topProduct


In [48]:
DBWriting(df_topCTR, df_topProduct)

(            advertiser_id product_id  click  impression  click-through-rate
 0    2WPF1NXECF3G6NUMWDO7     jarowj      1           9            0.111111
 1    2WPF1NXECF3G6NUMWDO7     10kluo      1          10            0.100000
 2    2WPF1NXECF3G6NUMWDO7     93h00e      1          10            0.100000
 3    2WPF1NXECF3G6NUMWDO7     o80z1x      1          10            0.100000
 4    2WPF1NXECF3G6NUMWDO7     dwknv4      1          13            0.076923
 ..                    ...        ...    ...         ...                 ...
 289  Y0W3K7OV6ZLILW96OO3K     1vk9yp      1          18            0.055556
 290  Y0W3K7OV6ZLILW96OO3K     b0x793      1          18            0.055556
 291  Y0W3K7OV6ZLILW96OO3K     ypztvs      1          20            0.050000
 292  Y0W3K7OV6ZLILW96OO3K     4xp2ib      1          25            0.040000
 293  Y0W3K7OV6ZLILW96OO3K     ur643h      1          26            0.038462
 
 [294 rows x 5 columns],
             advertiser_id product_id  cantidad
 

# DAG

In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

In [ ]:
# AGREGAR EL DS (seria el string de la fecha, habria que traducirlo a date)
# Aparte de pasarle el ds como argumento a la funcion, podriamos pasarle

In [ ]:
#Definimos nuestro DAG y sus tareas.

with DAG(
    dag_id = 'Recomendar',
    schedule_interval= '0 0 * * *', #se ejecuta a las 00:00 todos los días, todas las semanas, todos los meses
    start_date=datetime.datetime(2022,4,1),
    catchup=False
) as dag:

    FiltrarDatos = PythonOperator(
        task_id='Filtro',
        python_callable=FiltrarDatos, #función definida arriba
        op_kwargs = {"df_advertiser_ids" : df_advertiser_ids,
                    "df_product_views": df_product_views,
                    "df_ads_views",df_ads_views}
    )

    TopCTR = PythonOperator(
        task_id='TopCTR',
        python_callable=TopCTR, #función definida arriba
        op_kwargs = {"df_ads_views_filt" : df_ads_views_filt}
    )

    TopProduct = PythonOperator(
        task_id='TopProduct',
        python_callable=TopProduct, #función definida arriba
        op_kwargs = {"df_product_views_filt" : df_product_views_filt}
    )

    DBWriting = PythonOperator(
        task_id='DBWritting',
        python_callable=DBWritting, #función definida arriba
        op_kwargs = {"df_topCTR" : df_topCTR,
                     "df_topProduct" : df_topProduct}
    )

    

In [ ]:
#Dependencias
FiltrarDatos >> TopCTR
FiltrarDatos >> TopProduct
[TopCTR, TopProduct] >> DBWriting

# FastAPI

In [ ]:
@app.get("recommendations/{ADV}/{Modelo}") 
#devuelve un JSON en dónde se indican las recomendaciones del día para el adv y el modelo en cuestión.

@app.get("/stats/") 
#devuelve un JSON con un resumen de estadísticas sobre las recomendaciones
    # Cantidad de advertisers
    # Advertisers que más varían sus recomendaciones por día
    # Estadísticas de coincidencia entre ambos modelos para los diferentes advs.

@app.get("/history/<ADV>/") 
#devuelve un JSON con todas las recomendaciones para el advertirse pasado por parámetro en los últimos 7 días.

In [ ]:
import boto3
import pandas as pd

Si el bucket es púbico funciona, si el bucket está asociado a una cuenta en AWS hay que hacer el log previo.

In [ ]:
s3 = boto3.client("s3") #definimos un cliente para trabajar con S3 usando boto3
bucket_name = "annageller" #el nombre de nuestro bucket creado
s3_object = "sales/customers.csv" #el archivo que vamos a traernos

obj = s3.get_object(Bucket = bucket_name, Key=s3_object) #definimos el archivo a levantar
df = pd.read_csv(obj['Body']) #levantamos el DF

In [ ]:
s3 = boto3.client('s3')
bucket_name = 'nombre_del_bucket'
file_path = 'ruta_al_archivo_csv'
object_name = 'nombre_del_archivo_csv_en_el_bucket'

with open(file_path, 'rb') as file:
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=file)